In [49]:
#import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import math
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


Initialization and abstraction of data

In [50]:
def loadData():
    with np.load("notMNIST.npz") as data:
        Data, Target = data["images"], data["labels"]
        np.random.seed(521)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data = Data[randIndx] / 255.0
        Target = Target[randIndx]
        trainData, trainTarget = Data[:10000], Target[:10000]
        validData, validTarget = Data[10000:16000], Target[10000:16000]
        testData, testTarget = Data[16000:], Target[16000:]
    return trainData, validData, testData, trainTarget, validTarget, testTarget


In [51]:
def convertOneHot(trainTarget, validTarget, testTarget):
    newtrain = np.zeros((trainTarget.shape[0], 10))
    newvalid = np.zeros((validTarget.shape[0], 10))
    newtest = np.zeros((testTarget.shape[0], 10))

    for item in range(0, trainTarget.shape[0]):
        newtrain[item][trainTarget[item]] = 1
    for item in range(0, validTarget.shape[0]):
        newvalid[item][validTarget[item]] = 1
    for item in range(0, testTarget.shape[0]):
        newtest[item][testTarget[item]] = 1
    return newtrain, newvalid, newtest


In [52]:
def shuffle(trainData, trainTarget):
    np.random.seed(421)
    randIndx = np.arange(len(trainData))
    target = trainTarget
    np.random.shuffle(randIndx)
    data, target = trainData[randIndx], target[randIndx]
    return data, target



In [53]:
trainData, validData, testData, trainTarget, validTarget, testTarget= loadData()
train_y, valid_y, test_y = convertOneHot( trainTarget, validTarget, testTarget)
#Train Data 
train_x = trainData.reshape(10000,784)
train_X = train_x.T
#X0 = np.ones((3500,1))
#train_X = np.append(X0,train_X,axis=1)

#Test Data
global test_X
test_X = testData.reshape(2724,784)


#Validation Data
global validation_X
validation_X = validData.reshape(6000,784)
print(train_X.shape)

(784, 10000)


Initialization of weights and biases

In [54]:
#weights and bias from input to hidden layer ---> W, b
mu = 0
sigma =1 /  math.sqrt(784 + 1000)
W = np.random.normal(mu, sigma, [784,1000])
bW = np.zeros((1,1000))


#weights and bias from hidden to output layer ---> V,bV
mu = 0
sigma =1/ math.sqrt(1000 + 10)
V = np.random.normal(mu, sigma, [1000,10])
bV = np.zeros((1,10))
#print(V.shape)
#print(bV.shape)

Helper Functions

In [65]:
def relu(x) : 
    return np.maximum(x,0)

In [188]:
def softmax(z) :
    b = np.amax(z,axis=0)
    print("z:",z)
    print("b:",b)
    #print(b)
    #print(z-b)
    #exp = np.exp(z - np.max(b))
    e_pow = np.exp(z-np.max(z))
    print('e_pow',e_pow)
    #print(e_pow)
    summation = np.sum(e_pow)
    print('Summation :',summation)
    out = np.divide(e_pow,summation)
    print('out:',out)
    #out = exp / np.sum(exp) 
    return out

In [189]:
def computeLayer(X,W,b) :
    return np.dot(W.T,X) + b.T

In [190]:
### need to check inf * 0 problem
def CE(target, prediction):
    N = target.shape[0]
    #check assignment anouncement
    ### bug ###
    #prediction[prediction == 0] = 0.0000001
    log_term = np.log(prediction)
    CE_out = np.multiply(target,log_term)
    CE_out = CE_out
    CE_out = np.sum(CE_out) / N
    return -1*CE_out

In [11]:
def gradCE(target, prediction):

    # TODO


SyntaxError: unexpected EOF while parsing (<ipython-input-11-c12b5ed612cf>, line 3)

Forward Propagation 

In [191]:
#forward propagationForward Propagation

def forward_pass(train_X,W,bW,V,bV) :
    S1 = computeLayer(train_X,W,bW)
    #print("S1 >> ",S1.shape)
    #print(S1)
    Y = relu(S1)
    #print(Y)
    #print("Y >> ",Y.shape)
    S2 = computeLayer(Y,V,bV)
    #print(S2)
    #print("S2 >> ",S2.shape)
    Z = softmax(S2)
    #print(Z)
    #print("Z >> ",Z.shape)

    #test = np.sum(Z,0)
    #for i in range(10000) :
    #    if(test[i] != 1) :
    #        print(test[i])

    Z_1 = Z.T

    print("S1:",S1)
    print("S2:",S2)
    print("Y:",Y)
    print("Z:",Z)
    loss_out = CE(train_y, Z_1)
    return loss_out,S1,Y,S2,Z,Z_1


All Gradients

In [158]:
#### Gradient wrt outer layer weights 
#dL/dZ = [dL/ds2] * [ds2/dZ]
# dL/dV(i,j) = [ dL/ds2(j) ] * [ ds2(j)/dV(i,j) ]
# tot_Wo        = term_1 * term_2
# dL/ds2(j) = [ dL/dZ(j) ] * [ dZ(j)/dS2(j) ]
# term_1     = term_1_1 * term_1_2
def grad_wo(Y,Z_1,train_y,S2):
    term_2 = np.copy(Y)
    
    #print("term_2 >> ",term_2.shape)
    out = -(train_y / Z_1)
    term_1_1 = -1*(1/10000)*np.sum((train_y / Z_1),0)
    #print("term_1_1 >> ",term_1_1)
    

                

    b = np.amax(S2,axis=0)
    e_pow = np.exp(S2-b)
    summation = np.sum(e_pow,0)
    term_1_2 =  np.divide(e_pow,summation)
    #print("term_1_2 >> ",term_1_2.shape)
    
    ###########################
    ###### problem with o1
    ###### value large
    ###### multiplying by 1/10000 thats wrong
    ###### applies for all the gradient calculations
    ###########################
    o1 = np.matmul(term_1_2,term_2.T)
    #print("o1 >> ",o1)
    term_1_1_1 = term_1_1.reshape(10,1)
    #print("term_1_1_1 >> ",term_1_1_1)
    tot_Wo = np.multiply(o1.T,term_1_1_1.T)
    
    return (1/10000)*tot_Wo
    #print(tot_Wo)

In [159]:

#### Gradient wrt outer layer bias 
# dL/dV(i,j) = [ dL/ds2(j) ] * [ ds2(j)/dVb(i,j) ]
# tot_Wo        = term_1 * 1
# dL/ds2(j) = [ dL/dZ(j) ] * [ dZ(j)/dS2(j) ]
# term_1     = term_1_1 * term_1_2
def grad_bo(Z_1,S2,train_y):

    term_2 = np.ones((1,10000))


    out = (train_y / Z_1)
    term_1_1 = -1*(1/10000)*np.sum((train_y / Z_1),0)

    b = np.amax(S2,axis=0)
    e_pow = np.exp(S2-b)
    summation = np.sum(e_pow,0)
    term_1_2 =  np.divide(e_pow,summation)


    o1 = np.matmul(term_1_2,term_2.T)

    term_1_1_1 = term_1_1.reshape(10,1)
    tot_bo = np.multiply(o1.T,term_1_1_1.T)
    return (1/10000)*tot_bo

In [160]:
#### Gradient wrt hidden layer weight 
# dL/dW(i,j) = [ dL/ds1(j) ] * [ ds1(j)/dW(i,j) ]
# tot_Wh        = term_1 * term_2
def grad_wh(train_X,Y,tot_Wo,V):

    term_2 = np.copy(train_X)
    # dL/ds1(j) = [ dL/dY(j) ] * [ dY(j)/dS1(j) ]
    # term_1     = term_1_1 * term_1_2


    # dY(j)/dS1(j) = differentiation of relu 
    term_1_2 = np.copy(Y)
    term_1_2[term_1_2 > 0] = 1
    # dL/dY(j) = sumation([dL/dS2] * [dS2/dY])
    # term_1_1 = term_1_1_1 * term_1_1_2

    term_1_1_1 = np.copy(tot_Wo)
    term_1_1_2 = np.copy(V)
    ########
    ##check (1/1000)
    ########
    term_1_1 = np.sum(term_1_1_1*term_1_1_2,1)
    o1 = np.matmul(term_2,term_1_2.T)
    term_1_1_up = term_1_1.reshape(1000,1)
    tot_Wh = np.multiply(term_1_1_up.T,o1)
    return (1/10000)*tot_Wh

In [161]:
#### Gradient wrt hidden layer bias 
# dL/dW(i,j) = [ dL/ds1(j) ] * [ ds1(j)/db(i,j) ]
# tot_bh        = term_1 * term_2
def grad_bh(Y,tot_Wo,V):

    term_2 = np.ones((1,10000))
    # dL/ds1(j) = [ dL/dY(j) ] * [ dY(j)/dS1(j) ]
    # term_1     = term_1_1 * term_1_2

    # dY(j)/dS1(j) = differentiation of relu 
    term_1_2 = np.copy(Y)
    term_1_2[term_1_2 > 0] = 1
    # dL/dY(j) = sumation([dL/dS2] * [dS2/dY])
    # term_1_1 = term_1_1_1 * term_1_1_2

    term_1_1_1 = np.copy(tot_Wo)
    term_1_1_2 = np.copy(V)

    term_1_1 = np.sum(term_1_1_1*term_1_1_2,1)

    o1 = np.matmul(term_2,term_1_2.T)
    term_1_1_up = term_1_1.reshape(1000,1)
    tot_bh = np.multiply(term_1_1_up.T,o1)
    return (1/10000)*tot_bh

In [192]:
def back_propogation(train_y, Z_1, S2, Y, train_X):  
        
    #exp_S2 = np.exp(S2)
    #sums = np.sum(exp_S2, axis = 1)
    #Z= Z_1.T
    #print(Z.shape)
    #jacobian_m = np.zeros(S2.shape)
    #print("jacobian_m :",jacobian_m.shape)

    #for i in range(len(jacobian_m)):
     #   for j in range(len(jacobian_m)):
      #      if i == j:
       #         jacobian_m[i][j] = S2[i] * (1 - S2[i])
        #    else: 
         #       jacobian_m[i][j] = -1 * S2[i] * S2[j]
                
                
    #term1 = - np.divide(train_y, Z_1) #dervation of Loss Function
    #term2 =  jacobian_m  # term2 = derivation of softmax funtion
    
    #delta_1 = term1 * term2
    
    delta_1 = np.subtract(Z_1, train_y)
    
    #print('Delta_1 :',delta_1)
    
    #Derivative of Relu Function
    term_1_2 = np.copy(Y)
    term_1_2[term_1_2 > 0] = 1
    
    term3 = term_1_2
    #print("term_3", term3)
    
    #print("delta_1.shape",delta_1.shape)
    #print("V.shape",V.shape)
    #print("term3.shape",term3.shape)
    
    delta_temp = np.matmul(delta_1,V.T)
    delta_2 = np.multiply(delta_temp, term3.T)  # term3 = derivation of Relu function 
    #print("delta_2.shape",delta_2.shape)
    
    #dL/dV = delta_1 * Y
    dL_dV = np.matmul(Y, delta_1)    
    
    #dL/dW = delta_1 * train_X
    dL_dW = np.matmul(train_X, delta_2)
    
    return dL_dV, dL_dW, delta_1, delta_2
    
    

In [193]:
mV = np.full((1000,10),0.00001)
mW = np.full((784,1000),0.00001)
mbV = np.full((1,10),0.00001)
mb = np.full((1,1000),0.00001)
gamma = 0.9
alpha = 0.01


mu = 0
sigma =1 /  math.sqrt(784 + 1000)
W = np.random.normal(mu, sigma, [784,1000])
bW = np.zeros((1,1000))


#weights and bias from hidden to output layer ---> V,bV
mu = 0
sigma =1/ math.sqrt(1000 + 10)
V = np.random.normal(mu, sigma, [1000,10])
bV = np.zeros((1,10))

for i in range(500) :
    loss_out,S1,Y,S2,Z,Z_1 = forward_pass(train_X,W,bW,V,bV)
    #print("X.train",train_X.shape)
    #print("Y.shape",Y.shape)
    #print("S1.shape",S1.shape)
    #print("S2.shape",S2.shape)
    #print("Z.shape",Z.shape)
    print(loss_out)    

    tot_Wo,tot_Wh, delta_1, delta_2 = back_propogation(train_y, Z_1, S2, Y, train_X)
    #tot_Wo=grad_wo(Y,Z_1,train_y,S2)
    #print(tot_Wo)
    #print("-------")
    mV = gamma*mV + alpha*tot_Wo
    #print(V)
    #print("-------")
    V = V - mV
    bV = bV - gamma*1*delta_1
    #print(V)
    #tot_bo=grad_bo(Z_1,S2,train_y)
    #mbV = gamma*mbV + alpha*tot_bo
    #bV = bV - mbV
    
    #tot_Wh=grad_wh(train_X,Y,tot_Wo,V)
    mW = gamma*mW + alpha*tot_Wh
    W = W - mW
    bW = bW - gamma*1*delta_2
    #tot_bh=grad_bh(Y,tot_Wo,V)
    #mb = gamma*mb + alpha*tot_bh
    #bW = bW - mb
    

z: [[ -1.55580774e-03   2.62446258e-01  -1.01745643e-03 ...,  -1.64661982e-02
    4.75564524e-02   2.55411905e-03]
 [  2.55195787e-01   1.43866171e-01   2.22136154e-01 ...,   1.08787189e-01
    4.94540722e-01   1.10850639e-01]
 [ -2.20965309e-02   2.83306088e-02   9.77039656e-02 ...,   2.17939085e-01
    2.23578220e-01  -5.59661436e-02]
 ..., 
 [ -4.17065589e-01  -2.43814744e-01  -4.04371234e-01 ...,  -2.60267663e-01
   -7.85792377e-02  -9.51950145e-02]
 [  5.03793080e-03   2.02539743e-01   2.61996972e-01 ...,   1.10295514e-01
   -2.18823329e-01   1.01717268e-01]
 [ -1.74999492e-01   2.62124301e-01   1.11934006e-01 ...,  -1.03104121e-04
    1.13011141e-01  -5.39401295e-02]]
b: [ 0.28470309  0.26907864  0.31873353 ...,  0.25432852  0.49454072
  0.26786283]
e_pow [[ 0.40061679  0.52165549  0.40083252 ...,  0.39468775  0.42078314
   0.40226668]
 [ 0.51788692  0.46332433  0.50104569 ...,  0.44735315  0.65793218
   0.44827719]
 [ 0.39247177  0.4127705   0.44242241 ...,  0.49894717  0.501768

C:\Users\Urmi Joshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
C:\Users\Urmi Joshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in multiply
  


z: [[  1.15425646e+10   1.40081657e+10   1.08915929e+10 ...,   9.66740539e+09
    7.79094125e+09   3.52557026e+09]
 [  1.66526799e+10   2.01730577e+10   1.57179548e+10 ...,   1.39241520e+10
    1.12373336e+10   5.07627604e+09]
 [  1.16389701e+10   1.40809326e+10   1.09836128e+10 ...,   9.71940661e+09
    7.85089028e+09   3.54371592e+09]
 ..., 
 [  1.36139688e+10   1.64904076e+10   1.28507001e+10 ...,   1.13841125e+10
    9.18712740e+09   4.15024458e+09]
 [  1.70053103e+10   2.06289678e+10   1.60385054e+10 ...,   1.42326434e+10
    1.14734587e+10   5.18977603e+09]
 [  1.08948847e+10   1.32437190e+10   1.02956360e+10 ...,   9.13333867e+09
    7.35731380e+09   3.32891915e+09]]
b: [  1.70053103e+10   2.06289678e+10   1.60385054e+10 ...,   1.42326434e+10
   1.14734587e+10   5.18977603e+09]
e_pow [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]

KeyboardInterrupt: 